# New p2n API

## Pop2net only

In [11]:
import pop2net as p2n

env = p2n.Environment()

location1 = p2n.Location()
env.add_location(location1)

agent1 = p2n.Agent()
agent2 = p2n.Agent()
env.add_agents([agent1, agent2])

location1.add_agents([agent1, agent2])

## Pop2net + Agentpy

In [16]:
import agentpy as ap

class AgentpyAgent(p2n.Agent, ap.Agent):
    def say_hello(self):
        print("hello")

class AgentpyModel(ap.Model):
    def setup(self):
        self.env = p2n.Environment(model=self, framework="agentpy")
        
        location1 = p2n.Location()
        self.env.add_location(location1)

        agent1 = AgentpyAgent(model=self)
        agent2 = AgentpyAgent(model=self)
        self.env.add_agents([agent1, agent2])

        location1.add_agents([agent1, agent2])
    
    def step(self):
        self.env.agents.say_hello() # env enables agentlist usage

model = AgentpyModel()
model.run(steps=10)
    

hello
hello
Completed: 1 stepshello
hello
Completed: 2 stepshello
hello
Completed: 3 stepshello
hello
Completed: 4 stepshello
hello
Completed: 5 stepshello
hello
Completed: 6 stepshello
hello
Completed: 7 stepshello
hello
Completed: 8 stepshello
hello
Completed: 9 stepshello
hello
Completed: 10 steps
Run time: 0:00:00.000538
Simulation finished


DataDict {
'info': Dictionary with 9 keys
'reporters': DataFrame with 1 variable and 1 row
}

## Pop2net + Mesa

In [24]:
import mesa

class MesaAgent(p2n.Agent, mesa.Agent):
    def say_hello(self):
        print("hello")

class MesaModel(mesa.Model):
    def __init__(self):
        super().__init__()
        
        self.env = p2n.Environment(model=self, framework="mesa")
        
        location1 = p2n.Location()
        self.env.add_location(location1)

        agent1 = MesaAgent(model=self)
        agent2 = MesaAgent(model=self)
        self.env.add_agents([agent1, agent2])

        location1.add_agents([agent1, agent2])
    
    def step(self):
        self.env.agents.shuffle_do("say_hello") # env enables mesa's AgentSet usage

model = MesaModel()

for i in range(10):
    model.step()

hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello
hello


# Simulations using Pop2net

## No framework

In [1]:
import pop2net as p2n
import random

class OpinionAgent(p2n.Agent):
    def __init__(self, opinion):
        super().__init__()
        self.opinion = opinion
    
    def add_connection_to_other_cluster(self):
        not_neighbors = [agent for agent in self.env.agents if agent.opinion != self.opinion]
        self.connect(random.choice(not_neighbors))
    
class OpinionCluster(p2n.LocationDesigner):
    def split(self, agent):
        return agent.opinion

class Model:
    def __init__(self):
        self.env = p2n.Environment()
        self.env.add_agents([OpinionAgent(opinion = i%5) for i in range(100)])
        self.env.creator.create_locations(location_designers=[OpinionCluster])
    
    def step(self):
        for agent in self.env.agents:
            agent.add_connection_to_other_cluster()
    

model = Model()
for _ in range(3):
    model.step()
model.env.inspector.plot_agent_network()


## Agentpy

In [2]:
import pop2net as p2n
import agentpy as ap

class OpinionAgent(p2n.Agent, ap.Agent):
    def setup(self, opinion):
        self.opinion = opinion
    
    def add_connection_to_other_cluster(self):
        not_neighbors = self.env.agents.select(self.env.agents.opinion != self.env.agents.opinion) # we can use ap.agentLists now!
        self.connect(self.model.random.choice(not_neighbors))
    
class OpinionCluster(p2n.LocationDesigner):
    def split(self, agent):
        return agent.opinion

class Model(ap.Model):
    def setup(self):
        self.env = p2n.Environment(model=self, framework="agentpy")
        self.env.add_agents([OpinionAgent(model=self, opinion=i%5) for i in range(100)])
        self.env.creator.create_locations(location_designers=[OpinionCluster])
    
    def step(self):
        self.env.agents.add_connection_to_other_cluster() # we can use ap.agentLists now!

model = Model()
model.run(steps=3)
model.env.inspector.plot_agent_network()

Completed: 3 steps
Run time: 0:00:00.115003
Simulation finished


## Mesa

In [ ]:
import pop2net as p2n
import mesa

class OpinionAgent(p2n.Agent, mesa.Agent):
    def __init__(self, opinion, model):
        super().__init__(model)
        self.opinion = opinion
    
    def add_connection_to_other_cluster(self):
        not_neighbors = [agent for agent in self.env.agents if agent.opinion != self.opinion]
        self.connect(self.model.random.choice(not_neighbors))
    
class OpinionCluster(p2n.LocationDesigner):
    def split(self, agent):
        return agent.opinion

class Model(mesa.Model):
    def __init__(self, *args, seed = None, rng = None, **kwargs):
        super().__init__(*args, seed=seed, rng=rng, **kwargs)
        self.env = p2n.Environment(model=self, framework="mesa")
        self.env.add_agents([OpinionAgent(model=self, opinion=i%5) for i in range(100)])
        self.env.creator.create_locations(location_designers=[OpinionCluster])
    
    def step(self):
        self.env.agents.shuffle_do("add_connection_to_other_cluster")

model = Model()
for _ in range(3):
    model.step()
model.env.inspector.plot_agent_network()

## some tests

In [4]:
import pop2net as p2n

env = p2n.Environment()

agent = p2n.Agent()
env.add_agent(agent)

agent2 = p2n.Agent()
env.add_agent(agent2)

location = p2n.Location()
env.add_location(location)

location.add_agent(agent)
location.add_agent(agent2)

In [5]:
agent.neighbors()

In [6]:
env.locations

In [7]:
env.g.neighbors(location.id_p2n)

In [8]:
agent.locations

In [9]:
location.agents